# Import Required Libraries
Import the necessary libraries, such as requests for making HTTP requests and json for handling JSON data.

In [ ]:
# Import Required Libraries
import requests
import json
import pandas as pd
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

# Goal:

ich fahre von xxx nach Kassel und halte es maximal 1 h ohne WC aus. Schaffe ich das bei der aktuellen Verkehrslage?

# Fetch some information about Germany

- https://bund.dev/apis
 - pip install deutschland
- Dashboard Deutschland: https://dashboard.deutschland.api.bund.dev/
- Autbobahnen: https://autobahn.api.bund.dev/
- Deutschlandatlas: https://github.com/bundesAPI/deutschlandatlas-api



In [ ]:
url_bab = "https://verkehr.autobahn.de/o/autobahn"

# Get Roadworks
def roadworks(autobahn_id):
    url = f"{url_bab}/{autobahn_id}/services/roadworks"
    response = requests.get(url)
    return response.json()["roadworks"]

# Get parking_lorries
def parking_lorries(autobahn_id):
    url = f"{url_bab}/{autobahn_id}/services/parking_lorry"
    response = requests.get(url)
    return response.json()["parking_lorry"]

def get_warnings(autobahn_id):
    url = f"{url_bab}/{autobahn_id}/services/warning"
    response = requests.get(url)
    return response.json()["warning"]

autobahn_id = "A7"
json_roadwork = roadworks(autobahn_id)
json_data_parking = parking_lorries(autobahn_id)
json_data_warnings = get_warnings(autobahn_id)
# print(pd.DataFrame(json_data)


## now I create a class with a collection of 

In [ ]:
class TrafficWarning:
    def __init__(self, data):
        """
        Initialize the TrafficEvent class with data from the dictionary.

        :param data: dict, containing traffic event information
        """
        self.isBlocked = data.get('isBlocked')
        self.display_type = data.get('display_type')
        self.subtitle = data.get('subtitle')
        self.title = data.get('title')
        self.startTimestamp = data.get('startTimestamp')
        self.delayTimeValue = data.get('delayTimeValue')
        self.abnormalTrafficType = data.get('abnormalTrafficType')
        self.averageSpeed = data.get('averageSpeed')
        self.description = data.get('description')
        self.routeRecommendation = data.get('routeRecommendation')
        self.lorryParkingFeatureIcons = data.get('lorryParkingFeatureIcons')
        self.geometry = data.get('geometry')
        self.geo_df = self.create_geo_dataframe(self.geometry['coordinates'])

    def create_geo_dataframe(self, coordinates):
        df = pd.DataFrame(
            {'lat': [coord[0] for coord in coordinates],
             'long': [coord[1] for coord in coordinates]}
            )
        
        return df



traffic_events = [TrafficWarning(data) for data in json_data_warnings]

In [ ]:
# plot the traffic
def map_plot(plotlist, on='aveg_speed'):
    on_stats = pd.concat(df[on] for df in plotlist).describe()
    colormap = folium.LinearColormap(['green', 'yellow', 'red'], vmin=on_stats["min"], vmax=on_stats["max"])

    m = folium.Map(pd.concat([data[['long', 'lat']] for data in plotlist]).mean(), zoom_start=10)
    for df in plotlist:
        points_ = [(point[0], point[1]) for point in df[['long', 'lat']].to_numpy()]
        folium.features.ColorLine(
            points_,
            colors=[data for data in df[on]],
            colormap = colormap,
            weight=5,
            smooth_factor=3,
        ).add_to(m)
    return m

plot_data = [data.geo_df.assign(aveg_speed=float(data.averageSpeed)) for data in traffic_events if data.averageSpeed is not None]

map_plot(plot_data, on='aveg_speed')